# Areios Pagos Crawling

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
from pprint import pprint

import pandas as pd
import datetime
import logging
from tqdm import tqdm
import json
import time
import re
import os

In [30]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("extraction_process.log"),
        logging.StreamHandler()
    ]
)

In [3]:
def open_firefox():
        firefox_options = Options()
        firefox_options.add_argument('--headless')
        driver_path = r'C:\Program Files\geckodriver-v0.35.0-win64\geckodriver.exe'
        service = Service(driver_path)
        driver = webdriver.Firefox(service=service, options=firefox_options)
        return driver

In [4]:
def search_decisions(driver, year="2024"):
        driver.get("https://areiospagos.gr/nomologia/apofaseis.asp")
        wait = WebDriverWait(driver, 10)

        # fill 2024 as the year to be searched
        year_input = wait.until(expected_conditions.presence_of_element_located((By.NAME, 'x_ETOS')))
        year_input.clear()
        year_input.send_keys(year)

        # select 'ΟΛΕΣ' stis 'Αποφάσεις'
        select_tmhma = Select(driver.find_element(By.NAME, 'X_TMHMA'))
        select_tmhma.select_by_value('6')

        # select 'ΟΛΕΣ' sto 'Τμήμα'
        select_sub_tmhma = Select(driver.find_element(By.NAME, 'X_SUB_TMHMA'))
        select_sub_tmhma.select_by_value('1')

        search_button = driver.find_element(By.NAME, 'submit_krit')
        search_button.click()

        time.sleep(2)
        # print(driver.page_source)
        return driver.page_source

# Usage
driver = open_firefox()
html_content = search_decisions(driver, "2024") 
print(html_content)

<html><head>
<meta http-equiv="Content-Type" content="text/html; charset=windows-1253">
<!--<meta name="robots" content="noindex"%>-->
<title>ΑΡΕΙΟΣ ΠΑΓΟΣ - ΑΠΟΤΕΛΕΣΜΑΤΑ ΑΝΑΖΗΤΗΣΗΣ</title>
<link rel="stylesheet" type="text/css" href="../css/style1.css">
</head>

<body bgcolor="#EFF7FF">

<p style="margin-top: 0; margin-bottom: 0">


</p><p align="center"><a href="apofaseis.asp" class="blue10_cursor" onclick="history.back();return false;">&lt;&lt; Επιστροφή</a>	</p>
		<u><b><font face="Verdana" size="2">Αποτελέσματα αναζήτησης:</font></b></u><p></p>
		
<font color="#000099">
<b>&nbsp;&nbsp;&nbsp; <br>&nbsp;<br>

ΠΟΛΙΤΙΚΕΣ  -  Β1<hr color="blue">
</b></font>
 						
				<table border="0" width="100%" style="margin: 0 2; " height="38" cellspacing="1" cellpadding="5">
				<tbody><tr>

				<td width="30" height="0" valign="top">
					<div class="black10" align="center">
					1)   
					</div>
				</td>
				
				<td style="margin-top: 0; margin-bottom: 0" width="100" height="0" valign="top">

In [ ]:
def extract_decision_links(driver, html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    links = []

    decision_links = soup.select('a.blue10_cursor[href^="apofaseis_DISPLAY.asp"]')
    total_links = len(decision_links)
    logging.info(f"Found {total_links} decision links to process.")

    for idx, a_tag in enumerate(decision_links, 1):
        try:
            decision_id = a_tag.text.strip()
            info = a_tag.get('href').split('info')[1] if 'info=' in a_tag.get('href') else ""
            href = a_tag.get('href')
            url = f"https://areiospagos.gr/nomologia/{href}"

            logging.info(f"[{idx}/{total_links}] Fetching decision ID: {decision_id} from URL: {url}")
            driver.get(url)
            time.sleep(1)  # optional delay
            html_source = driver.page_source

            links.append({
                'decision_id': decision_id,
                'info': info,
                'url': url,
                'html_source': html_source
            })

        except Exception as e:
            logging.error(f"Error fetching decision at index {idx}: {str(e)}")

    logging.info(f"Finished processing {len(links)} decision links.")
    return links

links = extract_decision_links(driver, html_content)
output_filename = 'html_decisions.json'

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(links, f, ensure_ascii=False, indent=2)

logging.info(f"Saved {len(links)} decisions to {output_filename}")

2025-05-04 16:26:42,493 - INFO - Found 2475 decision links to process.
2025-05-04 16:26:42,493 - INFO - [1/2475] Fetching decision ID: 11/2024 from URL: https://areiospagos.gr/nomologia/apofaseis_DISPLAY.asp?cd=RWKDTA2X1554EH8YXW0M7NFLRRV23P&apof=11_2024&info=ΠΟΛΙΤΙΚΕΣ -  Β1
2025-05-04 16:26:43,971 - INFO - [2/2475] Fetching decision ID: 12/2024 from URL: https://areiospagos.gr/nomologia/apofaseis_DISPLAY.asp?cd=N1RABGMSWJWJ7DHP4GZ8JI6Q4TFTMD&apof=12_2024&info=ΠΟΛΙΤΙΚΕΣ -  Β1
2025-05-04 16:26:45,278 - INFO - [3/2475] Fetching decision ID: 13/2024 from URL: https://areiospagos.gr/nomologia/apofaseis_DISPLAY.asp?cd=RSMY30KIH82MLOOC1VPLFAXXXSO0UO&apof=13_2024&info=ΠΟΛΙΤΙΚΕΣ -  Β1
2025-05-04 16:26:46,691 - INFO - [4/2475] Fetching decision ID: 14/2024 from URL: https://areiospagos.gr/nomologia/apofaseis_DISPLAY.asp?cd=ZU5LE0TC6T0315OGAQ6IMDLESK4ST3&apof=14_2024&info=ΠΟΛΙΤΙΚΕΣ -  Β1
2025-05-04 16:26:48,695 - INFO - [5/2475] Fetching decision ID: 31/2024 from URL: https://areiospagos.gr/nomo

KeyboardInterrupt: 

In [8]:
def extract_decision_number_year(decision_text):
    decision_match = re.search(r'Απόφαση\s+(\d+)\s*/\s*(\d{4})', decision_text)
    number = decision_match.group(1) if decision_match else "Unknown"
    year = decision_match.group(2) if decision_match else "Unknown"
    return number, year

In [ ]:
def extract_department_info(decision_text):
    dept_match = re.search(r'\((.*?),\s*(.*?)\)', decision_text)
    if dept_match:
        dept_code = dept_match.group(1)
        dept_type = dept_match.group(2)
        
        if "ΠΟΙΝΙΚΕΣ" in dept_type:
            dept_type = "Ποινικό"
        elif "ΠΟΛΙΤΙΚΕΣ" in dept_type:
            dept_type = "Πολιτικό"
    else:
        dept_match_alt = re.search(r'(Β\d+\'?\s*[Π|π]ολιτικό\s*Τμήμα)', decision_text)
        if dept_match_alt:
            dept_parts = dept_match_alt.group(1).split()
            dept_code = dept_parts[0] if dept_parts else "Unknown"
            dept_type = "Πολιτικό" if "πολιτικό" in dept_match_alt.group(1).lower() else "Unknown"
        else:
            dept_code = "Unknown"
            dept_type = "Unknown"
    
    return dept_code, dept_type

In [10]:
def extract_judges_names(decision_text):
    judges_match = re.search(
        r'(?:Συγκροτήθηκε|ΣΥΓΚΡΟΤΗΘΗΚΕ) από τους [Δδ]ικαστές[:,]?\s*(.+?)Αρεοπαγίτες\.',
        decision_text,
        re.DOTALL
    )
    
    if not judges_match:
        return 'Unknown'
    
    judges_names = re.sub(r'\([^)]*\)', '', judges_match.group(1))
    
    titles_to_remove = [
        'Αντιπρόεδρο του Αρείου Πάγου,', 'Προεδρεύουσα Αρεοπαγίτη,', 'Προεδρεύουσα Αρεοπαγίτη ,',
        'Προεδρεύοντα Αρεοπαγίτη ,', 'Προεδρεύοντα Αρεοπαγίτη,', 'Εισηγητή,', 'Εισηγητή ,', 'Εισηγητής,'
    ]
    
    for title in titles_to_remove:
        judges_names = judges_names.replace(title, '')
    
    judges_names = re.sub(r'\s+', ' ', judges_names.replace(' και', ', ')).strip(', ')
    return judges_names

In [11]:
def extract_section_content(decision_text, start_phrases, end_phrases):
    if isinstance(start_phrases, str):
        start_phrases = [start_phrases]
    
    pattern = "(" + "|".join([re.escape(p) for p in start_phrases]) + r")\s*(.+?)(?=" + \
              "|".join([re.escape(p) for p in end_phrases]) + "|$)"
    
    match = re.search(pattern, decision_text, re.DOTALL | re.IGNORECASE)
    return re.sub(r'\s+', ' ', match.group(0).strip()) if match else "Unknown"

In [12]:
def extract_document_sections(decision_text):
    court_section = extract_section_content(
        decision_text,
        ["ΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ΠΑΓΟΥ", "ΤΟ ΔΙΚΑΣΤΗ ΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ΠΑΓΟΥ"],
        ["ΣΚΕΦΘΗΚΕ ΣΥΜΦΩΝΑ ΜΕ ΤΟ ΝΟΜΟ", "ΓΙΑ ΤΟΥΣ ΛΟΓΟΥΣ ΑΥΤΟΥΣ"]
    )
    
    reasoning_section = extract_section_content(
        decision_text,
        ["ΣΚΕΦΘΗΚΕ ΣΥΜΦΩΝΑ ΜΕ ΤΟ ΝΟΜΟ", "ΣΚΕΦΘΗΚΕ ΣΥΜΦΩΝΑ ΜΕ ΤΟΝ ΝΟΜΟ"],
        ["ΓΙΑ ΤΟΥΣ ΛΟΓΟΥΣ ΑΥΤΟΥΣ"]
    )
    
    decision_section = extract_section_content(
        decision_text,
        "ΓΙΑ ΤΟΥΣ ΛΟΓΟΥΣ ΑΥΤΟΥΣ",
        ["Ο ΑΝΤΙΠΡΟΕΔΡΟΣ", "Η ΑΝΤΙΠΡΟΕΔΡΟΣ", "Η ΓΡΑΜΜΑΤΕΑΣ"]
    )
    
    return court_section, reasoning_section, decision_section

In [13]:
def extract_articles_text(text):
    patterns = [
        re.compile(r"\b(άρθρ(?:ο|ου|α|ων)\s*\d+(?:[Α-Ωα-ω])?(?:\s+(?:παρ\.?|αριθ\.?|αρ\.?)\s*\d+)?" + 
                  r"(?:\s*(?:εδ\.?|περ\.?|στοιχ\.?)\s*[α-ωΑ-Ω'\"]+)?(?:\s+του\s+(?:Ν\.\s*\d+(?:/\d+)?|ΚΠολΔ|KΠολΔ|ΑΚ|ΠΚ|ΚΠΔ))?)", 
                  re.IGNORECASE),
        re.compile(r"\b(άρθρ(?:ο|ου|α|ων)\s*\d+(?:[Α-Ωα-ω])?\s*έως\s*\d+(?:[Α-Ωα-ω])?(?:\s+του\s+" + 
                  r"(?:Ν\.\s*\d+(?:/\d+)?|ΚΠολΔ|KΠολΔ|ΑΚ|ΠΚ|ΚΠΔ))?)", 
                  re.IGNORECASE),
        re.compile(r"\b(άρθρ(?:ο|ου|α|ων)\s*\d+(?:[Α-Ωα-ω])?((?:\s*,\s*\d+(?:[Α-Ωα-ω])?)+(?:\s*,\s*\d+(?:[Α-Ωα-ω])?\s*" + 
                  r"(?:παρ\.?\s*\d+)?(?:\s*(?:εδ\.?|περ\.?|στοιχ\.?)\s*[α-ωΑ-Ω'\"]+)?)*(?:\s+(?:και|και του|του)\s+" + 
                  r"(?:\d+(?:[Α-Ωα-ω])?))?\s+του\s+(?:Ν\.\s*\d+(?:/\d+)?|ΚΠολΔ|KΠολΔ|ΑΚ|ΠΚ|ΚΠΔ))?)", 
                  re.IGNORECASE)
    ]
    
    matches = [m.group(0) for p in patterns for m in p.finditer(text)]
    unique = list(dict.fromkeys([re.sub(r'\s+', ' ', m.strip()) for m in matches]))
    
    return ", ".join(unique) if unique else "Unknown"

In [14]:
def extract_single_articles(text, dept_type, code_types):
    pattern = re.compile(r"\b(άρθρ(?:ο|ου|α|ων)\s*(\d+)(?:([Α-Ωα-ω]))?(?:\s+(?:παρ\.?|αριθ\.?|αρ\.?)\s*(\d+))?" + 
                        r"(?:\s*(?:εδ\.?|περ\.?|στοιχ\.?)\s*([α-ωΑ-Ω'\"]+))?(?:\s+του\s+(?:(Ν\.)\s*(\d+)(?:/(\d+))?|(ΚΠολΔ|KΠολΔ|ΑΚ|ΠΚ|ΚΠΔ)))?)", 
                        re.IGNORECASE)
    
    structured_articles = []
    
    for match in pattern.finditer(text):
        full_match, article_num, letter_suffix, paragraph, section, law_prefix, law_num, law_year, code = match.groups()

        article_code = None
        law_reference = None
        

        if code and code.upper() in ["ΚΠΟΛΔ", "KΠΟΛΔ", "ΑΚ", "ΠΚ", "ΚΠΔ"]:
            article_code = code

        elif law_prefix and law_num:
            law_reference = f"{law_prefix}{law_num}/{law_year if law_year else ''}"

        elif code_types:
            article_code = code_types[0]

        article_entry = {
            "type": "article",
            "article_number": article_num,
            "letter_suffix": letter_suffix if letter_suffix else None,
            "paragraph": paragraph if paragraph else None,
            "section": section if section else None,
            "code": article_code,
            "original_text": full_match
        }

        if law_reference:
            article_entry["law_reference"] = law_reference
            
        structured_articles.append(article_entry)
    
    return structured_articles

In [15]:
def extract_article_ranges(text, dept_type, code_types):
    pattern = re.compile(r"\b(άρθρ(?:ο|ου|α|ων)\s*(\d+)(?:([Α-Ωα-ω]))?\s*έως\s*(\d+)(?:([Α-Ωα-ω]))?(?:\s+του\s+" + 
                        r"(?:(Ν\.)\s*(\d+)(?:/(\d+))?|(ΚΠολΔ|KΠολΔ|ΑΚ|ΠΚ|ΚΠΔ)))?)", 
                        re.IGNORECASE)
    
    structured_articles = []
    
    for match in pattern.finditer(text):
        full_match, start_num, start_suffix, end_num, end_suffix, law_prefix, law_num, law_year, code = match.groups()
        
        article_code = None
        law_reference = None
        

        if code and code.upper() in ["ΚΠΟΛΔ", "KΠΟΛΔ", "ΑΚ", "ΠΚ", "ΚΠΔ"]:
            article_code = code

        elif law_prefix and law_num:
            law_reference = f"{law_prefix}{law_num}/{law_year if law_year else ''}"

        elif code_types:
            article_code = code_types[0]
        
        article_entry = {
            "type": "range",
            "start_number": start_num,
            "start_suffix": start_suffix if start_suffix else None,
            "end_number": end_num,
            "end_suffix": end_suffix if end_suffix else None,
            "code": article_code,
            "original_text": full_match
        }
        
        if law_reference:
            article_entry["law_reference"] = law_reference
            
        structured_articles.append(article_entry)
    
    return structured_articles

In [16]:
def extract_article_lists(text, dept_type, code_types):
    pattern = re.compile(r"\b(άρθρ(?:ο|ου|α|ων)\s*(\d+)(?:([Α-Ωα-ω]))?((?:\s*,\s*(\d+)(?:([Α-Ωα-ω]))?)+)(?:\s+του\s+" + 
                        r"(?:(Ν\.)\s*(\d+)(?:/(\d+))?|(ΚΠολΔ|KΠολΔ|ΑΚ|ΠΚ|ΚΠΔ)))?)", 
                        re.IGNORECASE)
    
    structured_articles = []
    
    for match in pattern.finditer(text):
        groups = match.groups()
        full_match, first_num, first_suffix, comma_list = groups[:4]
        law_info = groups[6:]
        
        law_prefix, law_num, law_year, code = law_info

        article_code = None
        law_reference = None
        
        if code and code.upper() in ["ΚΠΟΛΔ", "KΠΟΛΔ", "ΑΚ", "ΠΚ", "ΚΠΔ"]:
            article_code = code

        elif law_prefix and law_num:
            law_reference = f"{law_prefix}{law_num}/{law_year if law_year else ''}"
        elif code_types:
            article_code = code_types[0] 
        
        additional_nums = re.findall(r'(\d+)(?:([Α-Ωα-ω]))?', comma_list)

        article_entry = {
            "type": "list",
            "primary_number": first_num,
            "primary_suffix": first_suffix if first_suffix else None,
            "additional_numbers": [{"number": num, "suffix": suffix if suffix else None} for num, suffix in additional_nums],
            "code": article_code,
            "original_text": full_match
        }
        
        if law_reference:
            article_entry["law_reference"] = law_reference
            
        structured_articles.append(article_entry)
    
    return structured_articles

In [17]:
def extract_structured_articles(text, dept_type):
    code_types_map = {
        "Πολιτικό": ["ΚΠολΔ", "ΑΚ"],
        "Ποινικό": ["ΠΚ", "ΚΠΔ"]
    }

    code_types = code_types_map.get(dept_type, [])
    
    single_articles = extract_single_articles(text, dept_type, code_types)
    article_ranges = extract_article_ranges(text, dept_type, code_types)
    article_lists = extract_article_lists(text, dept_type, code_types)
    
    return single_articles + article_ranges + article_lists

In [18]:
def extract_numeric_dates(text):
    pattern = r'(\d{1,2})[./\-](\d{1,2})[./\-](\d{4})'
    matches = re.finditer(pattern, text)
    
    dates = []
    for match in matches:
        day, month, year = match.groups()
        try:
            if 1 <= int(day) <= 31 and 1 <= int(month) <= 12 and len(year) == 4:
                dates.append(f"{day}-{month}-{year}")
        except ValueError:
            continue
    
    return dates

In [19]:
def extract_textual_dates(text):
    greek_months = {
        'Ιανουαρίου': '01', 'Φεβρουαρίου': '02', 'Μαρτίου': '03', 'Απριλίου': '04',
        'Μαΐου': '05', 'Ιουνίου': '06', 'Ιουλίου': '07', 'Αυγούστου': '08',
        'Σεπτεμβρίου': '09', 'Οκτωβρίου': '10', 'Νοεμβρίου': '11', 'Δεκεμβρίου': '12',
        'Ιανουάριος': '01', 'Φεβρουάριος': '02', 'Μάρτιος': '03', 'Απρίλιος': '04',
        'Μάιος': '05', 'Ιούνιος': '06', 'Ιούλιος': '07', 'Αύγουστος': '08',
        'Σεπτέμβριος': '09', 'Οκτώβριος': '10', 'Νοέμβριος': '11', 'Δεκέμβριος': '12'
    }
    
    month_names = '|'.join(greek_months.keys())
    pattern = rf'(\d{{1,2}})\s+({month_names})\s+(\d{{4}})'
    
    dates = []
    matches = re.finditer(pattern, text, re.IGNORECASE)
    
    for match in matches:
        day, month_name, year = match.groups()
        try:
            if 1 <= int(day) <= 31 and len(year) == 4:
                month_num = greek_months.get(month_name, '00')
                day_formatted = day.zfill(2)
                dates.append(f"{day_formatted}-{month_num}-{year}")
        except (ValueError, KeyError):
            continue
    
    return dates

In [20]:
def extract_all_dates(text):
    numeric_dates = extract_numeric_dates(text)
    textual_dates = extract_textual_dates(text)

    all_dates = numeric_dates + textual_dates
    unique_dates = []
    seen = set()
    
    for date in all_dates:
        if date not in seen:
            seen.add(date)
            unique_dates.append(date)

    try:
        sorted_dates = sorted(unique_dates, key=lambda x: datetime.strptime(x, "%d-%m-%Y"))
        return sorted_dates
    except:
        return unique_dates

In [21]:
def extract_decision_details_from_html(decision_id, url, html_source):
    try:
        soup = BeautifulSoup(html_source, 'html.parser')
        decision_text = soup.get_text()

        number, year = extract_decision_number_year(decision_text)
        dept_code, dept_type = extract_department_info(decision_text)
        judges_names = extract_judges_names(decision_text)

        court_section, reasoning_section, decision_section = extract_document_sections(decision_text)

        articles_referenced = extract_articles_text(decision_text)
        structured_articles = extract_structured_articles(decision_text, dept_type)

        dates_mentioned = extract_all_dates(decision_text)

        return {
            'decision_id': decision_id,
            'decision_number': number,
            'decision_year': year,
            'department_code': dept_code,
            'department_type': dept_type,
            'judges_names': judges_names,
            'court_section': court_section,
            'reasoning_section': reasoning_section,
            'decision_section': decision_section,
            'articles_referenced': articles_referenced,
            'structured_articles': structured_articles,
            'dates_mentioned': dates_mentioned,
            'url': url
        }
    except Exception as e:
        logging.error(f"Error processing decision {decision_id}: {str(e)}")
        return {
            'decision_id': decision_id,
            'error': str(e),
            'url': url
        }

In [22]:
def prepare_articles_for_csv(structured_articles):

    article_count = 0
    range_count = 0
    list_count = 0
    
    article_numbers = []
    article_codes = []
    law_references = []
    
    for article in structured_articles:
        article_type = article.get('type', '')

        if article_type == 'article':
            article_count += 1
            article_num = article.get('article_number', '')
            if article.get('letter_suffix'):
                article_num += article.get('letter_suffix')
            article_numbers.append(article_num)
        elif article_type == 'range':
            range_count += 1
            start = article.get('start_number', '')
            if article.get('start_suffix'):
                start += article.get('start_suffix')
            end = article.get('end_number', '')
            if article.get('end_suffix'):
                end += article.get('end_suffix')
            article_numbers.append(f"{start}-{end}")
        elif article_type == 'list':
            list_count += 1
            nums = [article.get('primary_number', '')]
            if article.get('primary_suffix'):
                nums[0] += article.get('primary_suffix')
            
            for additional in article.get('additional_numbers', []):
                num = additional.get('number', '')
                if additional.get('suffix'):
                    num += additional.get('suffix')
                nums.append(num)
            
            article_numbers.append(",".join(nums))

        if article.get('code'):
            article_codes.append(article.get('code'))
        if article.get('law_reference'):
            law_references.append(article.get('law_reference'))
    
    article_summary = {
        'article_count': article_count,
        'range_count': range_count,
        'list_count': list_count,
        'total_articles': article_count + range_count + list_count,
        'article_numbers': "|".join(article_numbers) if article_numbers else "",
        'article_codes': "|".join(list(set(article_codes))) if article_codes else "",
        'law_references': "|".join(list(set(law_references))) if law_references else ""
    }
    
    return article_summary

In [33]:
def create_csv_from_json(json_data, output_file, trim_sections):
    flattened_data = []

    for item in json_data:
        if 'error' in item:
            continue

        flat_item = item.copy()

        if 'structured_articles' in flat_item:
            article_summary = prepare_articles_for_csv(flat_item['structured_articles'])
            flat_item.update(article_summary)
            del flat_item['structured_articles']

        if 'dates_mentioned' in flat_item and isinstance(flat_item['dates_mentioned'], list):
            flat_item['dates_mentioned'] = "|".join(flat_item['dates_mentioned'])

        if trim_sections:
            for section in ['court_section', 'reasoning_section', 'decision_section']:
                if section in flat_item and isinstance(flat_item[section], str):
                    flat_item[section] = flat_item[section][:200] + "..." if len(flat_item[section]) > 200 else flat_item[section]

        flattened_data.append(flat_item)

    df = pd.DataFrame(flattened_data)
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    return len(flattened_data)


In [34]:
def main():
    start_time = datetime.datetime.now()
    logging.info(f"Starting extraction process at {start_time}")
    
    input_file = "html_decisions.json"
    json_output_file = "legal_data_set.json"
    csv_output_file = "legal_data_set.csv"
    csv_output_file_readable = "legal_data_set_short_edition.csv"

    if not os.path.exists(input_file):
        logging.error(f"Input file {input_file} not found!")
        return
    
    try:
        with open(input_file, "r", encoding="utf-8") as f:
            decisions = json.load(f)
            logging.info(f"Loaded {len(decisions)} decisions from {input_file}")
    except Exception as e:
        logging.error(f"Error loading input file: {str(e)}")
        return
    
    results = []
    successful = 0
    failed = 0
    
    for item in tqdm(decisions, desc="Processing decisions"):
        try:
            result = extract_decision_details_from_html(
                item.get('decision_id', 'unknown'), 
                item.get('url', ''), 
                item.get('html_source', '')
            )
            results.append(result)
            
            if 'error' in result:
                failed += 1
                logging.warning(f"Failed to process decision {result['decision_id']}: {result['error']}")
            else:
                successful += 1

            if (successful + failed) % 100 == 0:
                with open(json_output_file, "w", encoding="utf-8") as f:
                    json.dump(results, f, ensure_ascii=False, indent=2)
                logging.info(f"Saved intermediate results after processing {successful + failed} decisions")
                
        except Exception as e:
            failed += 1
            logging.error(f"Unexpected error processing decision: {str(e)}")
    
    try:
        with open(json_output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        logging.info(f"Successfully saved {len(results)} records to {json_output_file}")
    except Exception as e:
        logging.error(f"Error saving JSON output file: {str(e)}")
    
    try:
        csv_count = create_csv_from_json(results, csv_output_file, trim_sections=False)
        create_csv_from_json(results, csv_output_file_readable, trim_sections=True)
        
        logging.info(f"Successfully saved {csv_count} records to {csv_output_file}")
    except Exception as e:
        logging.error(f"Error creating CSV file: {str(e)}")
    
    end_time = datetime.datetime.now()
    duration = end_time - start_time
    
    logging.info(f"Extraction process completed in {duration}")
    logging.info(f"Results: {successful} successful, {failed} failed, {len(results)} total")
    
    print(f"\nExtraction process completed:")
    print(f"- Duration: {duration}")
    print(f"- Successful: {successful}")
    print(f"- Failed: {failed}")
    print(f"- Total: {len(results)}")
    print(f"- JSON output saved to: {json_output_file}")
    print(f"- CSV output saved to: {csv_output_file}")
    print(f"- Log file: extraction_process.log")

if __name__ == "__main__":
    main()

2025-05-04 16:45:54,615 - INFO - Starting extraction process at 2025-05-04 16:45:54.615163
2025-05-04 16:45:56,530 - INFO - Loaded 2475 decisions from html_decisions.json
Processing decisions: 100%|██████████| 2475/2475 [04:08<00:00,  9.98it/s]
2025-05-04 16:50:14,634 - INFO - Successfully saved 2475 records to legal_data_set.json
2025-05-04 16:50:31,791 - INFO - Successfully saved 2475 records to legal_data_set.csv
2025-05-04 16:50:31,791 - INFO - Extraction process completed in 0:04:37.176272
2025-05-04 16:50:31,791 - INFO - Results: 2475 successful, 0 failed, 2475 total



Extraction process completed:
- Duration: 0:04:37.176272
- Successful: 2475
- Failed: 0
- Total: 2475
- JSON output saved to: legal_data_set.json
- CSV output saved to: legal_data_set.csv
- Log file: extraction_process.log
